In [361]:
import pdfplumber
import pandas as pd
import camelot
import tabula
import re
from dateutil import parser

def extract_text_from_pdf(pdf_file_path):
    with pdfplumber.open(pdf_file_path) as pdf:
        all_text = ""
        for page_number, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            all_text += text
    return all_text

def save_text_file(text , output_txt_file):
    with open(output_txt_file, "w" , encoding='utf-8') as text_file:
      text_file.write(text)

def extract_table(pdf, csv, type):
    tables = camelot.read_pdf(pdf, flavor='lattice', pages='all', split_text=True)
    df = pd.DataFrame()
    header_set = set() # to record the column name
    for i,table in enumerate(tables):
        table_df = table.df
        if not table_df.empty:
            first_row = tuple(table_df.iloc[0])
            header_set.add(first_row)
            table_df.columns = first_row
            if i>0:
                table_df.columns=df.columns #always set column names equal to the start page columns
            df = pd.concat([df,table_df], ignore_index=True, axis=0) #row-wise concat
    df = df.dropna(axis=0, how='all')
    df = df.dropna(axis=1, how='all')
    df = df[~df.apply(lambda row: all(row == df.columns), axis=1)] # remov repeating column name(if any)
    df.to_csv(csv, index=False)
    print("Tables saved successfully to : ", csv)
    
def extract_table_HDFC(pdf,csv):
    tables = tabula.read_pdf(pdf, pages='all', multiple_tables=True)
    if len(tables) > 0:
        df = pd.DataFrame()
        for i,table in enumerate(tables):
            table = table.dropna(axis=0, how='all')
            table = table.dropna(axis=1, how='all')
            first_row = table.columns
            table.columns = first_row
            if i>0:
                table.columns = df.columns
            df = pd.concat([df,table], ignore_index=True, axis=0)
    df = df.dropna(subset=[df.columns[0]])
    df.to_csv(csv, index=False)
    print("Tables saved successfully to : ",csv)

def main():
    pdf = 'HDFC.pdf'
    type = 'HDFC'
    text = 'text_output.txt'
    csv = 'output.csv'
    text_data = extract_text_from_pdf(pdf)
    save_text_file(text_data , text)
    if (type == 'HDFC'):
        extract_table_HDFC(pdf,csv)
    else:
        extract_table(pdf,csv,type)
main()

Tables saved successfully to :  output.csv


In [362]:

def details_extract(txt_file,type):
    with open(txt_file,'r') as file:
        text = file.read()
    # Remove (cid:n) occurrences where 'n' is any number
    text = re.sub(r'\(cid:\d+\)', '', text)
    text = text.lower()
    customer = r'(?:customer\s*:|customer name\s*:|account name\s*:|mr.\s*|mrs.\s*|miss\s*|ms.\s*|mas\s*)(.*)\n'
    email = r'(?:email\s*:\s*|email id\s*:\s*)(.*)\n'
    from_date = r'\b(?:from\s*:\s*|from\s*|for period\s*:\s*)(\d{1,2}[/\s-]\d{1,2}[/\s-]\d{2,4}|\d{1,2}\s[A-Za-z]{3}\s\d{2,4})'
    to_date = r'(?:to\s*:\s*|to:|to\s*)(\d{1,2}[/\s-]\d{1,2}[/\s-]\d{2,4}|\d{1,2} [A-Za-z]{3} \d{2,4})'
    
    customer_match = re.search(customer, text)
    customer_name = customer_match.group(1) if customer_match else None
    if customer_name:
        customer_name = customer_name.title()
    
    email_match = re.search(email, text)
    email_id = email_match.group(1) if email_match else None
    
    from_match = re.search(from_date, text)
    from_res = from_match.group(1) if from_match else None
    if from_res:
        from_res = from_res.title()
    
    to_match = re.search(to_date,text)
    to = to_match.group(1) if to_match else None
    if to:
        to = to.title()
    if (customer_name == None or re.search(r'[0-9]',customer_name)):
        customer_name = 'Name not found'
    if (email_id == None): 
        email_id = 'Email id not found'
    print('CUSTOMER NAME : ',customer_name)
    print('EMAIL ID : ',email_id)
    print('FROM DATE : ',from_res)
    print('TO DATE : ',to)
    
def main():
    txt_file = 'text_output.txt'
    type = 'HDFC'
    details_extract(txt_file,type)
main()

CUSTOMER NAME :  Rohanshriramgaikwad
EMAIL ID :  rohan1618033@gmail.com
FROM DATE :  11/08/2023
TO DATE :  11/09/2023


 CODE FOR STANDARDISING COLUMNS OF EXTRACTED TABLE 
 COLUMN NAMES : DATE , DESCRIPTION / ((NARRATION)) , AMOUNT (DR,CR) , BALANCE

In [ ]:

def rename_to_narration(df):
    for column in df.columns:
        try:
            threshold = 15
            has_large_alphanum = df[column].astype(str).str.count('[A-Za-z0-9]').max() >= threshold
            if has_large_alphanum:
                df = df.rename(columns = {column: 'Narration'},inplace=True)
        except(ValueError, TypeError):
            continue

def rename_to_date(df):
    for column in df.columns:
        try:
            if df[column].apply(lambda x: parser.parse(str(x), fuzzy=True, dayfirst=True)).notna().all():
                df = df.rename(columns = {column:'Date'},inplace=True)
        except(ValueError, TypeError):
            continue

def rename_to_balance(df):
    pattern = r'^[0-9,.]+$'
    for column in df.columns:
        try:    
            if df[column].apply(lambda x: isinstance(x, str) and bool(re.match(pattern, x)) if not pd.isna(x) else False).all():
                df.rename(columns = {column: 'Balance'},inplace = True)
        except(ValueError, TypeError):
            continue

def rename_credit_debit(df):
    for i in range(0,30):
        if i > 29:
            break
        df['Balance'] = df['Balance'].astype(str).str.replace(',', '', regex=True).astype(float)
        value = df['Balance'][i+1] - df['Balance'][i]
        if value > 0 : 
            for column in df.columns:
                try:
                    df[column] = df[column].astype(str).str.replace(',','',regex=True).astype(float)
                except (ValueError,TypeError):
                    continue
                if value == df[column][i+1]:
                    df.rename(columns = {column: 'Credit'},inplace = True)
        else:
            for column in df.columns:
                try:
                    df[column] = df[column].astype(str).str.replace(',','',regex=True).astype(float)
                except (ValueError,TypeError):
                    continue
                if value == -(df[column][i+1]):
                    df.rename(columns = {column: 'Debit'},inplace = True)

def standardise_table(df,type,csv):
    req_col = ['Date', 'Narration', 'Amount', 'Balance']
    
    if (type == 'HDFC' or type == 'SBI' or type == 'AXIS'):           
        df.fillna(0.0, inplace=True)
        df['Amount'] = df['Credit'] - df['Debit']
        df['Balance'] = df['Balance'].astype(str).str.replace(',', '', regex=True).astype(float)
        df['Date'] = df['Date'].apply(lambda x: parser.parse(x, dayfirst=True, fuzzy=True))
        df = df[req_col]
        df1 = df
        df1.to_csv(csv, index=False)
        
    elif (type == 'PNB'):
        df.fillna(0.0, inplace=True)
        df['Amount'] = df.apply(lambda row: -row['Amount'] if row['Type'] == 'DR' else row['Amount'], axis=1)
        df['Balance'] = df['Balance'].astype(str).str.replace(',', '', regex=True).astype(float)
        df['Date'] = df['Date'].apply(lambda x: parser.parse(x, dayfirst=True, fuzzy=True))
        df = df[req_col]
        df1 = df
        df1.to_csv(csv, index=False)

    elif (type == 'ICICI'):
        df.fillna(0.0, inplace=True)
        df['Amount'] = df.apply(lambda row: -row['Amount'] if row['Type'] == 'DR' else row['Amount'], axis=1)
        df['Date'] = df['Date'].apply(lambda x: parser.parse(x, dayfirst=True, fuzzy=True))
        df = df[['Date','Narration','Amount']]
        df1 = df
        df1.to_csv(csv, index=False)
    
    return df1

def main():
    csv = 'output.csv'
    type = 'HDFC'
    df = pd.read_csv(csv)
    df = df.dropna(subset=[df.columns[0]])
    rename_to_narration(df)
    rename_to_date(df)
    df = df.dropna(subset=['Date']).reset_index(drop=True)
    rename_to_balance(df)
    if(type == 'HDFC' or type == 'SBI' or type == 'AXIS'): 
        rename_credit_debit(df)
    df = standardise_table(df,type,csv)
    df.to_excel('output.xlsx', index=False, na_rep='N/A', header=custom_header, index_label='ID')
    print(df)
main()

          Date                                 Narration   Amount   Balance
0   2023-08-11  NEFT CR-YESB0000001-ZERODHA BROKING LTD-  1667.53  14748.63
1   2023-08-11   1008545301 TATA MOTORS LTD ORD DIV22 23     4.00  14752.63
2   2023-08-12  UPI-VRINDAVAN DAIRY FIRM-PAYTMQR28100505   -54.00  14698.63
3   2023-08-12  UPI-GIRNAR SWEETS-PAYTMQR2810050501012FC   -68.00  14630.63
4   2023-08-13  UPI-PALAVA TFG1-PAYTMQR28100505010163ZCU  -390.00  14240.63
..         ...                                       ...      ...       ...
116 2023-09-09  UPI-AMAZON SELLER SERVIC-AMAZONSELLERSER  -399.00  15981.13
117 2023-09-09  UPI-AMAZON SELLER SERVIC-AMAZONSELLERSER   399.00  16380.13
118 2023-09-11                       UPI-ZERODHA BROKING -1000.00  15380.13
119 2023-09-11   UPI-M R C CANTEEN-PAYTMQR281005050101BA   -30.00  15350.13
120 2023-09-11  UPI-BHARAT BAU RAJPUT-PAYTMQRU35R2Y9RXU@  -195.00  15155.13

[121 rows x 4 columns]
